# select conda envirnment coz of FAISS library

In [14]:
import pandas as pd

In [15]:
df=pd.read_csv('datasetUpdate.csv')

In [16]:
df = df[['title', 'overview', 'genre', 'release_date']]

In [17]:
df.describe()

,title,overview,genre,release_date
count,10000,9987,9997,10000
unique,9661,9985,2123,6113
top,Beauty and the Beast,"""Loro"", in two parts, is a period movie that c...",Comedy,11/4/2016
freq,4,2,744,9


In [18]:
df.columns

Index(['title', 'overview', 'genre', 'release_date'], dtype='object')

In [26]:
df=df[['title', 'overview', 'genre','release_date']]

In [27]:
df

,title,overview,genre,release_date
0,Inception,"Cobb, a skilled thief who commits corporate es...","Action,Science Fiction,Adventure",7/15/2010
1,Interstellar,The adventures of a group of explorers who mak...,"Adventure,Drama,Science Fiction",11/5/2014
2,The Dark Knight,Batman raises the stakes in his war on crime. ...,"Drama,Action,Crime,Thriller",7/14/2008
3,The Avengers,When an unexpected enemy emerges and threatens...,"Science Fiction,Action,Adventure",4/25/2012
4,Deadpool,The origin story of former Special Forces oper...,"Action,Adventure,Comedy",2/9/2016
...,...,...,...,...
9995,Splendor in the Grass,A fragile Kansas girl's unrequited and forbidd...,"Drama,Romance",10/10/1961
9996,Pillow Talk,A man and woman share a telephone line and des...,"Comedy,Romance",10/7/1959
9997,The 7th Voyage of Sinbad,"When a princess is shrunken by an evil wizard,...","Action,Adventure,Family,Fantasy",12/5/1958
9998,Tarantula,A lone-wolf scientist sequestered in a mansion...,"Horror,Science Fiction,Thriller",12/14/1955


In [21]:
# df['tags'] = df['overview']+df['genre']

In [22]:
# new_data = df.drop(columns=['overview', 'genre'])

In [23]:
# new_data

,title,release_date,tags
0,Inception,7/15/2010,"Cobb, a skilled thief who commits corporate es..."
1,Interstellar,11/5/2014,The adventures of a group of explorers who mak...
2,The Dark Knight,7/14/2008,Batman raises the stakes in his war on crime. ...
3,The Avengers,4/25/2012,When an unexpected enemy emerges and threatens...
4,Deadpool,2/9/2016,The origin story of former Special Forces oper...
...,...,...,...
9995,Splendor in the Grass,10/10/1961,A fragile Kansas girl's unrequited and forbidd...
9996,Pillow Talk,10/7/1959,A man and woman share a telephone line and des...
9997,The 7th Voyage of Sinbad,12/5/1958,"When a princess is shrunken by an evil wizard,..."
9998,Tarantula,12/14/1955,A lone-wolf scientist sequestered in a mansion...


In [28]:
# Create textual representation
def create_textual_representation(row):
    return f"""
Title: {row['title']},
Year: {row['release_date']},
Genre: {row['genre']},

Description: {row['overview']}"""

In [29]:
df_copy = df.copy()
df_copy['textual_representation'] = df_copy.apply(create_textual_representation, axis=1)

In [30]:
import faiss
print(faiss.__version__)
import requests
import numpy as np
# Initialize FAISS index
dim = 4096  # Ensure this matches the dimension of the embeddings returned by the LLM
index = faiss.IndexFlatL2(dim)

1.7.4


In [31]:
# Generate embeddings and add to FAISS index
x = np.zeros((len(df_copy['textual_representation']), dim), dtype='float32')

In [32]:
#x

In [33]:
for i, representation in enumerate(df_copy['textual_representation']):
    if i % 1000 == 0:
        print(f"Processing textual representation {i}")

Processing textual representation 0
Processing textual representation 1000
Processing textual representation 2000
Processing textual representation 3000
Processing textual representation 4000
Processing textual representation 5000
Processing textual representation 6000
Processing textual representation 7000
Processing textual representation 8000
Processing textual representation 9000


In [39]:
def get_embedding_from_api(representation, index):
    try:
        res = requests.post('http://localhost:11434/api/embeddings', 
                            json={'model': 'llama2', 'prompt': representation})
        res.raise_for_status()  # Raise an HTTPError for bad responses
        response_data = res.json()
        
        if 'embedding' in response_data:
            return np.array(response_data['embedding'], dtype='float32')
        else:
            print(f"Warning: No embedding key found in response for index {index}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error: Request failed for index {index}. Exception: {e}")
        return None

In [40]:
index.add(x)
faiss.write_index(index, 'index2')

In [41]:
# Load FAISS index and perform a search
index = faiss.read_index('index')

In [42]:
# Example recommendation
recommend_movie = """
Title: The Plot,
Year: 21 June 2024,
Genre: 

Description: The Plot is a 2024 South Korean crime thriller film directed by Lee Yo-sup, starring Gang Dong-won. An adaptation of the Hong Kong film, Accident, it tells the story of Yeong-il, a designer who orchestrates a commissioned murder as a perfect accidental death, and gets caught up in an unexpected incident
"""

In [43]:
res = requests.post('http://localhost:11434/api/embeddings', 
                    json={'model': 'llama2', 'prompt': recommend_movie})

In [44]:
embedding = np.array([res.json().get('embedding')], dtype='float32')
D, I = index.search(embedding, 5)

In [45]:
best_matches = np.array(df_copy['textual_representation'])[I.flatten()]
for match in best_matches:
    print('Next Movie')
    print(match)
    print()

Next Movie

Title: Interstellar,
Year: 11/5/2014,
Genre: Adventure,Drama,Science Fiction,

Description: The adventures of a group of explorers who make use of a newly discovered wormhole to surpass the limitations on human space travel and conquer the vast distances involved in an interstellar voyage.

Next Movie

Title: The Dark Knight,
Year: 7/14/2008,
Genre: Drama,Action,Crime,Thriller,

Description: Batman raises the stakes in his war on crime. With the help of Lt. Jim Gordon and District Attorney Harvey Dent, Batman sets out to dismantle the remaining criminal organizations that plague the streets. The partnership proves to be effective, but they soon find themselves prey to a reign of chaos unleashed by a rising criminal mastermind known to the terrified citizens of Gotham as the Joker.

Next Movie

Title: The Avengers,
Year: 4/25/2012,
Genre: Science Fiction,Action,Adventure,

Description: When an unexpected enemy emerges and threatens global safety and security, Nick Fury, dire

In [46]:
import pickle
# Save and load results
pickle.dump(best_matches, open('best_matches.pkl', 'wb'))
best_matches_loaded = pickle.load(open('best_matches.pkl', 'rb'))

In [47]:
print(best_matches_loaded)

['\nTitle: Interstellar,\nYear: 11/5/2014,\nGenre: Adventure,Drama,Science Fiction,\n\nDescription: The adventures of a group of explorers who make use of a newly discovered wormhole to surpass the limitations on human space travel and conquer the vast distances involved in an interstellar voyage.'
 '\nTitle: The Dark Knight,\nYear: 7/14/2008,\nGenre: Drama,Action,Crime,Thriller,\n\nDescription: Batman raises the stakes in his war on crime. With the help of Lt. Jim Gordon and District Attorney Harvey Dent, Batman sets out to dismantle the remaining criminal organizations that plague the streets. The partnership proves to be effective, but they soon find themselves prey to a reign of chaos unleashed by a rising criminal mastermind known to the terrified citizens of Gotham as the Joker.'
 '\nTitle: The Avengers,\nYear: 4/25/2012,\nGenre: Science Fiction,Action,Adventure,\n\nDescription: When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the 